<a href="https://www.kaggle.com/code/probalkar/diabetic-retinopathy-detection?scriptVersionId=171393618" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Loading dependencies

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

## Importing CSV dataset

In [ ]:
data = pd.read_csv("/kaggle/input/diabetic-retinopathy-resized/trainLabels_cropped.csv")
print(data.value_counts().sum())
data.head()

Adding image names with extension in the dataframe

In [ ]:
data_imgs = [data['image'][i] + '.jpeg' for i in range(len(data))]
data['image_name'] = data_imgs
data.head()

Dropping unnecessary columns from the dataframe

In [ ]:
data.drop(columns=['Unnamed: 0.1','Unnamed: 0', 'image'], inplace=True)
data.head()

Randomly selecting half the number of datapoints from the dataset to train and evaluate the model

In [ ]:
random_rows = data.sample(n=int(data.value_counts().sum()/2), random_state=42)
print(random_rows)

Splitting the dataset into training and validation set with 20% data for evaluation

In [ ]:
train, val = train_test_split(random_rows, test_size=0.2, random_state=42)

## Image preprocessing
Rescaling the images

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

Declaring constants

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5

Creating training and validation set of images

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/',
    x_col='image_name',
    y_col='level',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw')

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val,
    directory='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/',
    x_col='image_name',
    y_col='level',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw')

## MobileNetV2 Pre-trained model
Loading pre-trained model MobileNetV2 with ImageNet classification labels

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

Compiling the model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training the model

In [ ]:
model.fit(train_generator, validation_data=val_generator, epochs=30)